In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Welcome to the **stratosphere** dashboard!

**stratosphere** is a free and open source OSINT platform that automatically collects every page you visit, building a private knowledge base you can analyze with Jupyter notebooks and an extensible suite of web apps.

***

In [2]:
from stratosphere import Stratosphere
from stratosphere.services.extractor import get_list_extractors
from stratosphere import options
from IPython.display import HTML
from markdown import markdown
import os

warning_msg = """
> **WARNING**: No mounted volume detected on `/shared`, your knowledge base and modifications
are lost if you terminate the Docker container. This is perfectly fine to experiment and try the project.
"""

mounted_msg = """
Persistence of changes is **active**: You mounted a host directory on the `/shared` mount point.
"""

if os.path.exists('/shared/warning_no_mounted_volume'):
    msg = warning_msg
else:
    msg = mounted_msg
    
display(HTML(markdown(msg)))

def first_or_null(s):
    if len(s) >0:
        return s.iloc[0]
    else:
        return "N/A"


In [5]:
kb = Stratosphere(options.get("db.url_kb"))
probe = Stratosphere(options.get("db.url_probe"))
count_entities = kb.db.pandas("SELECT count(*) as cnt FROM entities").cnt.iloc[0]
count_relationships = kb.db.pandas("SELECT count(*) as cnt FROM relationships").cnt.iloc[0]
last_entity_ts = kb.db.pandas("SELECT ts FROM entities ORDER BY ts DESC LIMIT 1").ts.iloc[0]
last_relationship_ts = kb.db.pandas("SELECT ts FROM relationships ORDER BY ts DESC LIMIT 1").ts.iloc[0]
last_flow_ts = first_or_null(probe.db.pandas("SELECT flow_capture_timestamp FROM flows ORDER BY flow_capture_timestamp DESC LIMIT 1").flow_capture_timestamp)
size_mb_probe = probe.db.size()
size_mb_kb = kb.db.size()

registered_extractors = ""
for modname in get_list_extractors():
    registered_extractors += f"* {modname}\n"

mkd = f"""
Overview:

* The knowledge base contains **{count_entities}** entities and **{count_relationships}** relationships
* Timestamp of last inserted entity: **{last_entity_ts}**
* Timestamp of last inserted relationship: **{last_relationship_ts}**
* Timestamp of last intercepted flow: **{last_flow_ts}**
* Size (MiB) of probe.db: **{size_mb_probe:.2f}**
* Size (MiB) of kb.db: **{size_mb_kb:.2f}**

Registered extractors:

{registered_extractors}
"""

HTML(markdown(mkd))

/shared/data/probe.db
/shared/data/kb.db


Featured web apps:

* [vk.com contacts explorer](http://127.0.0.1:8082/jupyter/voila/render/webapps/vk.com-contacts-explorer.ipynb): Explore previously seen vk.com contacts, highlighting their connections
* [Google sarch results](http://127.0.0.1:8082/jupyter/voila/render/webapps/google-search-results.ipynb): Review your past Google search results

Other resources:

* [JupyterLab](/jupyter/lab): Create new Voilà web apps as notebooks, run ad-hoc analyses, and customize this page
* [Web apps directory](/jupyter/voila/tree/webapps): Access the complete list of web apps
* [SQLite web interface](/sqlite/): Experiment and query the underlying knowledge base with SQL

Recent system logs:


In [ ]:
!tail -10 /tmp/supervisord.log